In [30]:
# Concise Implementation of Linear Regression
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [31]:
# Reading the Dataset
# The boolean value 'is_train' indicates whether or not we want the data iterator object to shuffle the data on each epoch (pass through the dataset).
# next(iter()) 함수의 역할은 generator의 역할과 유사함을 기억할 필요가 있습니다.
def load_array(data_arrays, batch_size, is_train=True): #@save
    """"Construct a Tensorflow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [32]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.61314934,  0.38680825],
        [-1.297983  ,  1.3316035 ],
        [-0.3161761 , -2.056911  ],
        [ 0.30748326,  0.13282299],
        [-0.16749963, -1.0616336 ],
        [ 0.6440127 , -0.71006346],
        [ 0.09285521, -0.40607098],
        [ 0.05600452, -0.6347162 ],
        [ 0.2712449 , -0.81054944],
        [-0.30039108, -0.4910026 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 1.6609216],
        [-2.926529 ],
        [10.566331 ],
        [ 4.3503995],
        [ 7.4678593],
        [ 7.891633 ],
        [ 5.762725 ],
        [ 6.4689417],
        [ 7.503671 ],
        [ 5.2583838]], dtype=float32)>)

In [33]:
# Defining the Model
# We will first define a model variable 'net', which will refer to an instance of the 'Sequential' class.
# The 'Sequential' class defines a container for several layers that will be chained together.
# 'keras' is the high-level API for Tensorflow
# 일단은 직관적으로 어떤 의미인지만을 받아들이면서 넘어갑시다.
net = tf.keras.Sequential() # Feed-forward network의 일종으로 받아들이면 될 것 같습니다.
net.add(tf.keras.layers.Dense(1)) # Dense refers to the fully-connected layer structure. And 1 refers to the single-layer structure, i guess haha.

In [34]:
# Initializing Model Parameters
# Deep learning frameworks often have a predefined way to initialize the parameters.
# Here we specify that each weight parameter should be randomly sampled from a normal distn w/ mean 0 and sd 0.01.
# The bias parameter will be initialized to zero.

In [35]:
# The 'initializers' module in Tensorflow provides various methods for model parameter initialization.
# The easiest way to specify the initialization method in Keras is when creating the layer by specifying 'kernel_initializer'.
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer)) # One fantastic thing of using the keras API! Just be careful to remember that since the parameters have not been initialized yet, we cannot access or manipulate them.

In [36]:
# Defining the Loss Function
# The 'MeanSquaredError' class computes the mean squared error (w/o the 1/2 factor).
# By default, it returns the 'AVERAGE' loss over examples.4
loss = tf.keras.losses.MeanSquaredError()

In [37]:
# Defining the Optimization Algorithm
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

In [39]:
# Training
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
            #print(l)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables)) # weight랑 bias가 따로 들어가서 zip을 사용한 것 같습니다! 맞지요?!
        #print(grads)
        #print(net.trainable_variables)
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000106
epoch 2, loss 0.000107
epoch 3, loss 0.000106


In [40]:
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - b)

error in estimating w tf.Tensor([-6.1750412e-05 -6.6018105e-04], shape=(2,), dtype=float32)
error in estimating b [-0.00031185]
